In [1]:
import asyncio
from aioes import Elasticsearch

In [53]:
es = Elasticsearch(['localhost:9200'])

In [110]:
def gen_synonyms():
    """
    Generate some synonyms in a file. All words separated by comma are treated as equal
    """
    with open("synonyms.txt", "w") as syns:
        syns.write("xboost, эксгебуст, эксбуст, иксгебуст, xgboost\n")
        syns.write("пыха, пыху, пых, php\n")
        syns.write("lol, лол\n")
        syns.write("питон, python\n")

In [105]:
index_body = {
    "settings": {
        "analysis": {
          "filter": {
            "russian_stop": {
              "type":       "stop",
              "stopwords":  "_russian_" 
            },
            "russian_stemmer": {
              "type":       "stemmer",
              "language":   "russian"
            },
            "synonyms_expand": {
              "type": "synonym", 
              # path to synonym file.
              # for ES to be able to read it, security policy should be set as described here:
              # https://stackoverflow.com/questions/35401917/reading-a-file-in-an-elasticsearch-plugin
              "synonyms_path": "/Users/enchantner/Experiments/synonyms.txt"
            }
          },
          "analyzer": {
            "russian_syn": {
              "tokenizer":  "standard",
              "filter": [
                "lowercase",
                "russian_stop",
                "russian_stemmer",
                "synonyms_expand"
              ]
            }
          }
        }
    },
    "mappings":{  
        "question":{  
          "properties":{  
            "text":{"type":"string", "analyzer":"russian_syn"}
          }
        }
    }
}

async def create_index():
    ret = await es.indices.create(
        index="questions-index",
        body=index_body
    )
    print(ret)
    
    
async def delete_index():
    ret = await es.delete(
        index="questions-index"
    )
    print(ret)
    
async def openclose():
    """
    Closing and opening index again reloads synomyms file
    """
    await es.indices.close(index="questions-index")
    await es.indices.open(index="questions-index")
    
async def populate_index():
    await es.index(
        index="questions-index",
        doc_type="question",
        body={
            "text": "А что мне рассказать про иксгебуст?"
        }
    )
    await es.index(
        index="questions-index",
        doc_type="question",
        body={
            "text": "Я ненавижу PHP, что мне делать?"
        }
    )
    await es.index(
        index="questions-index",
        doc_type="question",
        body={
            "text": "Я люблю питон, что мне делать?"
        }
    )

In [103]:
loop = asyncio.get_event_loop()
loop.run_until_complete(delete_index())
loop.run_until_complete(create_index())
loop.run_until_complete(populate_index())

{'acknowledged': True}
{'acknowledged': True, 'shards_acknowledged': True}


In [111]:
# reload synonims without recreating the whole database
gen_synonyms()
loop.run_until_complete(openclose())